In [1]:
import numpy as np
import os
import pandas as pd
from ast import literal_eval
from pymongo import MongoClient
import utils.const as const
import re

In [2]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

['161136235596016',
 '1991716644473195',
 '3980924515308070',
 'admin',
 'config',
 'local']

In [3]:
db=const.grpId
collection=const.PostCollection
filename='javed.csv' 

In [4]:
myCollect=client[db][collection]

In [5]:
df = pd.read_csv(filename)

df.post_msg_tranlsated_final = df.post_msg_tranlsated_final.apply(literal_eval)
df.post_comments_translated_final = df.post_comments_translated_final.apply(literal_eval)
df.post_subcomments_translated_final = df.post_subcomments_translated_final.apply(literal_eval)

print(df.dtypes)
print(len(df.index))
df.head()

post_id                               object
post_msg_final                        object
post_comments_final                   object
post_subcomments_final                object
post_msg_tranlsated_final             object
post_comments_translated_final        object
post_subcomments_translated_final     object
Unnamed: 7                           float64
Unnamed: 8                           float64
Unnamed: 9                           float64
post_subcomments_auto                 object
dtype: object
18631


,post_id,post_msg_final,post_comments_final,post_subcomments_final,post_msg_tranlsated_final,post_comments_translated_final,post_subcomments_translated_final,Unnamed: 7,Unnamed: 8,Unnamed: 9,post_subcomments_auto
0,1991716644473195_2794406357537549,[{ '1991716644473195_2794406357537549': [ 'Tod...,"[{ ""2796316020679916"": [ """"]}, { ""279642353066...",[{ '2796502783994573': [ 'Jinal Patel thanks']...,[{' '1991716644473195_2794406357537549'': ['to...,"[{' ""2796423530669165""': ['nice abhinandana']}...",[{' '2796502783994573'': ['jinal patel thanks'...,NaN,NaN,NaN,[{ '2796502783994573': [ 'Jinal Patel thanks']...
1,1991716644473195_2794217790889739,[{ '1991716644473195_2794217790889739': [ 'Zan...,"[{ ""2796282750683243"": [ ""What's very abhinand...",[{ '2796333124011539': [ 'Tarul Patel need a v...,[{' '1991716644473195_2794217790889739'': ['za...,"[{' ""2796282750683243""': ['whats abhinandanaa ...",[{' '2796333124011539'': ['tarul patel need vi...,NaN,NaN,NaN,[{ '2796333124011539': [ 'Tarul Patel need a v...
2,1991716644473195_2796183594026492,[{ '1991716644473195_2796183594026492': [ 'Las...,"[{ ""2796308564013995"": [ """"]}, { ""279635010734...",[{ '2796462883998563': [ 'Janki Mangalani ok']...,[{' '1991716644473195_2796183594026492'': ['la...,"[{' ""2796350107343174""': ['exquisite']}, {' '2...",[{' '2796462883998563'': ['janki mangalani ok'...,NaN,NaN,NaN,[{ '2796462883998563': [ 'Janki Mangalani ok']...
3,1991716644473195_2795011284143723,"[{ ""19917166444731952795011284143723"": [ ""stre...","[{ '2796472850664233': [ '']}, { '279777969053...",[],"[{' ""1991716644473195_2795011284143723""': ['st...",[{' '2797779690533549'': ['good work']}],[],NaN,NaN,NaN,[]
4,1991716644473195_2792857774359074,[],"[{ '2797022117275973': [ 'Very nice, Congratul...",[],[],[{' '2797022117275973'': ['nice congratulation...,[],NaN,NaN,NaN,[]


In [6]:
for i in range(len(df.index)):
    if len(df['post_msg_tranlsated_final'][i]):
        for _,v1 in df['post_msg_tranlsated_final'][i][0].items(): ### its just contain one key,val pair
            postId = df['post_id'][i]
            
            m1=[]
            for t1 in v1:
                m1.extend(t1.split(' '))
            
            result=myCollect.update_one({'_id': postId}, {
                    "$set": {'message' : m1}
                })
            if result.matched_count==0 :
                print("=="+postId+"==")
                print(i,'fail1')
                print(result.matched_count , result.modified_count)
                break;
            
            for comment in df['post_comments_translated_final'][i]:
                for commentId1,v2 in comment.items():
                    commentId = re.sub('\D*', '', commentId1)
                    
                    m2=[]
                    for t2 in v2:
                        m2.extend(t2.split(' '))

                    result=myCollect.update_one({'_id': postId, 'comments': {'$elemMatch': {'_id': commentId}}}, {
                        '$set': {'comments.$.message':  m2}})
                    
                    if result.matched_count==0:
                        print("=="+commentId+"==")
                        print("=="+commentId1+"==")
                        print(i,'fail2')
                        print(result.matched_count , result.modified_count)
                        #break;
#                     elif result.modified_count==0:
#                         print("=="+commentId+"==")
#                         for x in myCollect.find({'_id': postId, 'comments': {'$elemMatch': {'_id': commentId}}}):
#                             print(x['message'])
                    
                    

In [7]:
c1=0
c2=0

for post in list(myCollect.find({})):
    if 'message' not in post:
        myCollect.update_one({'_id': post['_id']}, {
                    "$set": {'message' : ['']}
                })
    elif post['message']=='':
        myCollect.update_one({'_id': post['_id']}, {
                    "$set": {'message' : ['']}
                })
    elif not isinstance(post['message'], list):
        c1+=1;
        
    if 'comments' in post:
        for comment in post['comments']:
            if 'message' not in comment:
                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                        '$set': {'comments.$.message':  ['']}})
                
            elif comment['message']=='':
                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                        '$set': {'comments.$.message':  ['']}})
                
            elif not isinstance(comment['message'], list):
                c2+=1;
                

In [8]:
print(c1,c2)

754 12768
